In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
from tensorflow.keras.callbacks import TensorBoard

In [2]:
from sentence_transformers import SentenceTransformer

# Replace 'sentence-transformers/bert-base-nli-mean-tokens' with your desired model name
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
model = SentenceTransformer(model_name)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df2 = pd.read_parquet('clean_data.pqt')

In [4]:
df2

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww matches background colour seemingly stuck ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,make real suggestions improvement wondered sec...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,second time asking view completely contradicts...,0,0,0,0,0,0
159567,ffea4adeee384e90,ashamed horrible thing put talk page 128 61 19 93,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm theres actual article prostitution...,0,0,0,0,0,0
159569,fff125370e4aaaf3,looks like actually put speedy first version d...,0,0,0,0,0,0


In [5]:
df= df2.sample(400)

In [6]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
133579,ca9f898c7914b5f3,note moved text,0,0,0,0,0,0
30878,5207742f4ebf2fd3,please add commercial links links private webs...,0,0,0,0,0,0
71536,bf818d22b6f086c5,mike harper provided references suggest mike h...,0,0,0,0,0,0
50157,8609cd00bbe5554c,know real computer took real computer shop cou...,0,0,0,0,0,0
84947,e3588fe6714ecb03,season 3 info 3rd season 74 57 167 219,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
76155,cbd9987f157466c7,hello begin nobody works wikipedia volunteer p...,0,0,0,0,0,0
130037,b7a74a71dfd81339,support look http en wikipedia org wiki list_o...,0,0,0,0,0,0
69941,bb175180caa5d0e7,removal college material probably ok leave his...,0,0,0,0,0,0
131519,bfa7764bb9b34e6d,reverted recent edit made article mike tomlin ...,0,0,0,0,0,0


In [7]:
embeddings= model.encode(df['comment_text'].tolist())
df['emb'] = embeddings.tolist()

In [38]:
embeddings.shape

(400, 768)

In [16]:
max_len = df['emb'].apply(len).max()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [39]:
sequences = np.array(df['emb'].tolist())
sequences = sequences.reshape(400, 1, 768)
print("Shape of sequences before reshaping:", sequences.shape)

Shape of sequences before reshaping: (400, 1, 768)


In [40]:
max_words = 1300  # Number of embeddings per sample
dmsn = 768     # Dimensionality of each embedding


In [41]:


padded_sequences = pad_sequences(sequences, maxlen=max_words, padding='post', truncating='post')


In [42]:
X = np.array(padded_sequences)
y = df.drop(['id','comment_text', 'emb'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=77)

In [43]:
X_train.shape

(280, 1300, 768)

In [11]:
# dmsn = len(df['emb'].iloc[0])

# print("Dimensionality of the embeddings:", dmsn)

Dimensionality of the embeddings: 768


In [35]:
# # Reshape X_train and X_test to fit the RNN
# X_train = X_train.reshape(X_train.shape[0], max_words, dmsn)
# X_test = X_test.reshape(X_test.shape[0], max_words, dmsn)

ValueError: cannot reshape array of size 364000 into shape (280,1300,768)

In [ ]:
# filename= "ST_LSTM"
# tb = TensorBoard(log_dir= "logs\\{}".format(filename))

In [44]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

i = Input(shape=(max_words, dmsn))
x = LSTM(128)(i)
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='softmax')(x)
lstm_model2 = Model(i, x) 


In [45]:
lstm_model2.compile(loss='categorical_crossentropy', optimizer= Adam(learning_rate=.000025), metrics=['accuracy', Recall(name= 'recall'), Precision(name='precision')])
r =lstm_model2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
9/9 [==============================] - 6s 173ms/step - loss: 0.3520 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 1s 174ms/step - loss: 0.3520 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 1s 156ms/step - loss: 0.3520 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 1s 139ms/step - loss: 0.3519 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 5/10
9/9 [==============================] 

In [46]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Flatten

i = Input(shape=(max_words, dmsn))

# Conv1D layer
x = Conv1D(32, 3, activation='relu')(i)
x = MaxPooling1D(2)(x)

# Flatten layer
x = Flatten()(x)

# Dense layers
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='softmax')(x)

# Create the Model
cnn_model = Model(i,x)


In [47]:
cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(learning_rate=.000025), metrics=['accuracy', Recall(name= 'recall'), Precision(name='precision')])
g=cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
9/9 [==============================] - 4s 114ms/step - loss: 0.3516 - accuracy: 0.1821 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4327 - val_accuracy: 0.1583 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 1s 66ms/step - loss: 0.3514 - accuracy: 0.2357 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4326 - val_accuracy: 0.2333 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 1s 63ms/step - loss: 0.3511 - accuracy: 0.3286 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4323 - val_accuracy: 0.3417 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 1s 59ms/step - loss: 0.3507 - accuracy: 0.4821 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4319 - val_accuracy: 0.5750 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 5/10
9/9 [==============================] - 1

IndexError: tuple index out of range

In [ ]:
X_train

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0]])